In [9]:
# from statsmodels.discrete.discrete_model import MNLogit
# from statsmodels.tools.tools import add_constant
import numpy as np
import collections.abc
collections.Iterable = collections.abc.Iterable
from collections import OrderedDict
import pylogit as pl
import pandas as pd
import scipy.special

In [26]:
n_rows = 30000
n_teams = 20
n_candidates = 4
# randomly generate a n_rows and n_teams column numpy array where each row has exactly n_seats ones
X = np.zeros((n_rows, n_teams))
for i in range(n_rows):
    X[i, np.random.choice(n_teams, n_candidates, replace=False)] = 1

capabilities = np.concatenate([np.zeros(1), np.random.standard_normal(n_teams - 1)])
print(capabilities)
print(scipy.special.softmax(capabilities))

# for each row, randomly choose one as the winner
Y = np.zeros((n_rows))
for i in range(n_rows):
    indices = np.where(X[i, :] == 1)[0]
    prob = scipy.special.softmax(capabilities[indices])
    Y[i] = np.random.choice(indices, p=prob)
# convert the data to long format
L = np.zeros((n_rows * n_candidates, 3))
for i in range(n_rows):
    indices = np.where(X[i, :] == 1)[0]
    for j, idx in enumerate(indices):
        L[i*n_candidates + j, 0] = i
        L[i*n_candidates + j, 1] = idx
        L[i*n_candidates + j, 2] = 1 if Y[i] == idx else 0

# exog = X
# m = MNLogit(Y, exog)
# m_fit = m.fit(method='powell',maxiter=10000)

# print(scipy.special.softmax(capabilities[:n_candidates]))
# print(m_fit.predict(np.concatenate([np.ones(n_candidates), np.zeros(n_teams - n_candidates)])))

[ 0.          0.45115932  0.81491071 -0.25031573 -1.55576916  0.40856351
  0.26102905 -0.53848883  0.27563455  0.8477542   0.72776987  0.19864273
  0.17968097  1.76954774  0.0595302  -0.41408748  0.04676339 -0.50247251
 -0.50564258 -1.267766  ]
[0.03640069 0.05715387 0.08222821 0.02833994 0.00768153 0.05477048
 0.04725776 0.02124452 0.04795305 0.08497371 0.07536611 0.0443996
 0.04356564 0.21360649 0.03863343 0.02405879 0.03814334 0.02202361
 0.02195391 0.01024533]


In [27]:
# create a pandas dataframe from L
df = pd.DataFrame(L, columns=['round_id', 'team_id', 'win'])
specification = OrderedDict()
names = OrderedDict()
teams_except_the_first = list(range(1, n_teams))
specification["intercept"] = teams_except_the_first
names["intercept"] = [f'ASC_{i}' for i in teams_except_the_first]
mnl = pl.create_choice_model(data=df,
                                alt_id_col='team_id',
                                obs_id_col='round_id',
                                choice_col='win',
                                specification=specification,
                                model_type="MNL",
                                names=names)
mnl.fit_mle(np.zeros(n_teams - 1))
mnl.get_statsmodels_summary()


Log-likelihood at zero: -41,588.8308
Initial Log-likelihood: -41,588.8308


/home/arsdragonfly/neofoodclub/envs/default/lib/python3.10/site-packages/scipy/optimize/_minimize.py:555: RuntimeWarning: Method BFGS does not use Hessian information (hess).
  warn('Method %s does not use Hessian information (hess).' % method,


Estimation Time for Point Estimation: 1.06 seconds.
Final log-likelihood: -36,443.1869


<class 'statsmodels.iolib.summary.Summary'>
"""
                     Multinomial Logit Model Regression Results                    
===================================================================================
Dep. Variable:                         win   No. Observations:               30,000
Model:             Multinomial Logit Model   Df Residuals:                   29,981
Method:                                MLE   Df Model:                           19
Date:                     Tue, 03 Jan 2023   Pseudo R-squ.:                   0.124
Time:                             18:03:17   Pseudo R-bar-squ.:               0.123
AIC:                            72,924.374   Log-Likelihood:            -36,443.187
BIC:                            73,082.244   LL-Null:                   -41,588.831
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ASC_1          0.5055      0.042     12.117      0.000       0.424       0.587
ASC_2          0.8211      0.041     20.121      0.000       0.741       0.901
ASC_3         -0.1547      0.046     -3.384      0.001      -0.244      -0.065
ASC_4         -1.4809      0.065    -22.784      0.000      -1.608      -1.353
ASC_5          0.4629      0.042     11.105      0.000       0.381       0.545
ASC_6          0.3380      0.042      7.969      0.000       0.255       0.421
ASC_7         -0.4914      0.048    -10.163      0.000      -0.586      -0.397
ASC_8          0.3254      0.043      7.646      0.000       0.242       0.409
ASC_9          0.8919      0.041     21.901      0.000       0.812       0.972
ASC_10         0.7597      0.041     18.512      0.000       0.679       0.840
ASC_11         0.2939      0.042      6.918      0.000       0.211       0.377
ASC_12         0.2380      0.043      5.541      0.000       0.154       0.322
ASC_13         1.8718      0.041     45.748      0.000       1.792       1.952
ASC_14         0.0972      0.044      2.231      0.026       0.012       0.183
ASC_15        -0.3609      0.047     -7.652      0.000      -0.453      -0.268
ASC_16         0.0730      0.044      1.657      0.097      -0.013       0.159
ASC_17        -0.4626      0.048     -9.582      0.000      -0.557      -0.368
ASC_18        -0.4609      0.048     -9.526      0.000      -0.556      -0.366
ASC_19        -1.1646      0.058    -20.172      0.000      -1.278      -1.051
==============================================================================
"""

In [31]:
scipy.special.softmax(np.concatenate([np.zeros(1), mnl.coefs]))

array([0.03432093, 0.05689904, 0.07800863, 0.02940248, 0.0078058 ,
       0.05452371, 0.04812218, 0.02099694, 0.04752031, 0.08373164,
       0.0733655 , 0.04604697, 0.04354132, 0.22308306, 0.03782329,
       0.02392254, 0.03691993, 0.021609  , 0.02164674, 0.01071001])